# Under surveillance calibration

### Notebook configuration

In [106]:
import sys
import os
from collections import defaultdict

import numpy as np  # For matrix manipulation
import pandas as pd  # For output/input data processing
import matplotlib.pyplot as plt  # For visualizations
from csaps import csaps  # For smoothing splines
from scipy.optimize import minimize, Bounds, LinearConstraint # For optimization

sys.path.append(os.path.abspath("../src"))

import helpers

# Some aesthetic options
helpers.add_cell_timer()
np.set_printoptions(
    suppress=True, linewidth=300, formatter={"float": "{: 0.9f}".format}
)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

## Targets
### Poisson targets (counts with exposure)
| Item         |   MLH1           |     MSH2         | MSH6             |
|--------------|------------------|------------------|------------------|
| PY           |  12798 | 7961 | 2550 |
| CRC counts   |  167   | 93  | 12 |

### 10-year cumulative risks (value, 95% CI)
| Item         |   MLH1           |     MSH2         | MSH6             |
|--------------|------------------|------------------|------------------|
| Adenoma      | 32.2 (29.2–35.2) | 44.2 (40.0–48.4) | 38.4 (30.8–45.9) |
| Advanced     |  7.7 ( 6.0– 9.4) | 17.8 (14.6–21.0) |  9.4 ( 5.4–13.4) |
| CRC (10y)    | 11.3 ( 9.4–13.2) | 11.4 ( 8.9–14.0) |  4.7 ( 1.8– 7.7) |


In [96]:
genes = ["MLH1","MSH2","MSH6"]

# Person-years and CRC counts from Engel
PY  = {"MLH1":12798, "MSH2":7961, "MSH6":2550}
Ycrc= {"MLH1":  167, "MSH2":  93, "MSH6":  12}

# 10-year cumulative risks and 95% CI
aden = {
  "MLH1": {"p":0.322, "L":0.292, "U":0.352},
  "MSH2": {"p":0.442, "L":0.400, "U":0.484},
  "MSH6": {"p":0.384, "L":0.308, "U":0.459},
}
adv = {
  "MLH1": {"p":0.077, "L":0.060, "U":0.094},
  "MSH2": {"p":0.178, "L":0.146, "U":0.210},
  "MSH6": {"p":0.094, "L":0.054, "U":0.134},
}
crc10 = {
  "MLH1": {"p":0.113, "L":0.094, "U":0.132},
  "MSH2": {"p":0.114, "L":0.089, "U":0.140},
  "MSH6": {"p":0.047, "L":0.018, "U":0.077},
}

## Optimization code

### Initial parameter set

In [97]:
theta0 = {
    # transition rates (per year)
    "lambda_NS": {"MLH1": 0.038, "MSH2": 0.056, "MSH6": 0.048},   # N → S (initiation)
    "lambda_NSS": {"MLH1": 0.038, "MSH2": 0.056, "MSH6": 0.048}, # NS -> S (post S removal)
    "lambda_NAS": {"MLH1": 0.076, "MSH2": 0.112, "MSH6": 0.096}, # NA -> S (post A removal)
    "lambda_SA": {"MLH1": 0.01, "MSH2": 0.02, "MSH6": 0.012},   # S → A (growth)
    "lambda_AP": {"MLH1": 0.25, "MSH2": 0.25, "MSH6": 0.15},   # A → P (malignant transformation)
    "lambda_PC": {"MLH1": 0.8, "MSH2": 0.8, "MSH6": 0.8},   # P → C (clinical presentation)

    # optional: sex multipliers or gene ratios if you want fewer parameters
    "m_S0": {"MLH1": 0.02, "MSH2": 0.02, "MSH6": 0.02},        # miss at index for non-advanced adenoma
    "m_A0": {"MLH1": 0.01, "MSH2": 0.01, "MSH6": 0.01},        # miss at index for advanced adenoma
}

### Loss function helpers

In [98]:
EPS = 1e-9
def logit(p): 
    p = np.clip(p, EPS, 1 - EPS)
    return np.log(p / (1 - p))

def neglog_pois(y, mu):
    # drop log(y!) constant if you like
    return mu - y * np.log(mu + EPS)

def logit_gaussian_loss(p_hat, p, L, U):
    # p_hat from simulator; p, L, U are observed proportions in [0,1]
    eta_hat = logit(p_hat)
    eta     = logit(p)
    se      = (logit(U) - logit(L)) / (2 * 1.96 + EPS)
    return 0.5 * ((eta_hat - eta) / (se + EPS))**2 + np.log(se + EPS)

### Simulate engel study

In [99]:
genes = ["MLH1", "MSH2", "MSH6"]
sens = {"S": 0.75, "A": 0.90, "P": 0.99}    # colonoscopy sensitivities
followup = 10                               # years after index
dt = float(1/12)                            # simulation step in years (1 month)
n_steps = int(followup / dt) 

# Engel colo intervals in simulation           
intervals = {"MLH1": 1.56, "MSH2": 1.26, "MSH6": 1.43}  # colonoscopy intervals (total colo/PY)

def simulate_engel(theta):
    out_r_crc_py = {}
    out_p10_aden = {}
    out_p10_adv = {}
    out_p10_crc = {}
    
    # iterate by gene
    for g in genes:
        lamNS = theta["lambda_NS"][g]
        lamNSS = theta["lambda_NSS"][g]
        lamNAS = theta["lambda_NAS"][g]
        lamSA = theta["lambda_SA"][g]
        lamAP = theta["lambda_AP"][g]
        lamPC = theta["lambda_PC"][g]
        mS0 = theta["m_S0"][g]
        mA0 = theta["m_A0"][g]
        interval = intervals[g]
        
        N, NS, NA, S, A, P, C = (1 - mS0 - mA0), 0.0, 0.0, mS0, mA0, 0.0, 0.0
        
        # trackers
        t_since_scope = 0.0
        crc_count = 0.0
        py = 0.0
        ever_aden, ever_adv, ever_crc = 0.0, 0.0, 0.0
        
        for _ in range(n_steps):
            # ---- biology evolution over dt ----
            # transition probabilities per step
            p_NS = 1 - np.exp(-lamNS * dt)
            p_NSS = 1 - np.exp(-lamNSS * dt)
            p_NAS = 1 - np.exp(-lamNAS * dt)
            p_SA = 1 - np.exp(-lamSA * dt)
            p_AP = 1 - np.exp(-lamAP * dt)
            p_PC = 1 - np.exp(-lamPC * dt)
            
            # flows (Euler forward)
            dN = -N * p_NS
            dNS = -NS * p_NSS
            dNA = -NA * p_NAS
            dS = (N * p_NS) + (NS * p_NSS) + (NA * p_NAS) - (S * p_SA)
            dA = S * p_SA - A * p_AP
            dP = A * p_AP - P * p_PC
            dC = P * p_PC
            
            # update
            N += dN
            NS += dNS
            NA += dNA
            S += dS
            A += dA
            P += dP
            C += dC
            
            py += (N + NS + NA + S + A + P) * dt   # person-years alive in non-clinical states
            
            # Add interval CRCs to event totals and "ever" CRC risk
            crc_count += dC
            ever_crc += (1.0 - ever_crc) * dC  # cumulative probability logic
            
            t_since_scope += dt
            
            # ---- colonoscopy event ----
            if t_since_scope >= interval - 1e-6:
                # detection events
                det_S = sens["S"] * S
                det_A = sens["A"] * A
                det_P = sens["P"] * P

                # Update "ever" (1 - product(1 - p_i)) without MC
                ever_aden += (1.0 - ever_aden) * det_S
                ever_adv  += (1.0 - ever_adv)  * det_A
                ever_crc  += (1.0 - ever_crc)  * det_P
                
                # Count CRC events diagnosed at scope
                crc_count += det_P

                # remove detected lesions (return to N)
                NS += det_S 
                NA += det_A
                S -= det_S
                A -= det_A
                P -= det_P
                C += det_P

                t_since_scope = 0.0
        
        # CRC events = those who progressed to C during 10y
        ever_crc = ever_crc or (C > 0)

        # compute metrics
        out_r_crc_py[g] = crc_count / max(py, 1e-12)
        out_p10_aden[g] = float(np.clip(ever_aden, 0.0, 1.0))
        out_p10_adv[g]  = float(np.clip(ever_adv,  0.0, 1.0))
        out_p10_crc[g]  = float(np.clip(ever_crc,  0.0, 1.0))

    return out_r_crc_py, out_p10_aden, out_p10_adv, out_p10_crc

* `r_crc_py[g]` = model-implied CRC incidence rate per person-yr under surveillance
* `p10_aden[g]` = P(>= 1 adenoma detected within 10y after index)
* `p10_adv[g]` = P(>= 1 advanced adenoma detected within 10y after index)
* `p10_crc[g]` = P(CRC within 10y)

### Run simulation and calculate loss

In [ ]:
def engel_negloglik(theta):
    # 1) run ENGEL-specific simulation using intervals I_g (≈1.56, 1.26, 1.43 y)
    #    returns dicts keyed by gene:
    #    r_crc_py[g], p10_aden[g], p10_adv[g], p10_crc[g]
    r_crc_py, p10_aden, p10_adv, p10_crc = simulate_engel(theta)

    # Poisson piece: CRC counts over person-years
    pois = 0.0
    for g in genes:
        mu = PY[g] * np.clip(r_crc_py[g], EPS, None)
        pois += neglog_pois(Ycrc[g], mu)

    # Logit-normal pieces for 10-year risks
    logit_sum = 0.0
    for g in genes:
        logit_sum += logit_gaussian_loss(p10_aden[g], aden[g]["p"], aden[g]["L"], aden[g]["U"])
        logit_sum += logit_gaussian_loss(p10_adv[g],  adv[g]["p"],  adv[g]["L"],  adv[g]["U"])
        logit_sum += logit_gaussian_loss(p10_crc[g],  crc10[g]["p"], crc10[g]["L"], crc10[g]["U"])

    # Prior on dwell
    prior = 0.0
    prior_tau0, prior_sd, weight = 2.5, 1.0, 1.0
    for g in genes:
        lamPC = theta["lambda_PC"][g]
        dwell = 1.0 / lamPC
        logdiff = np.log(dwell / prior_tau0)
        prior += weight * 0.5 * (logdiff / prior_sd)**2

    loss = pois + logit_sum + prior
    print(f"poisson: {pois:.3f}   logit-norm: {logit_sum:.3f}   prior: {prior:.3f}   total: {loss:.3f}")
    return loss


In [121]:
loss0 = engel_negloglik(theta0)
print("Initial Engel loss:", loss0)


poisson -385.16686784994283

logit-norm 1020.2031458638879

prior 1020.9238253847651
Initial Engel loss: 1020.9238253847651


### Optimization setup and multistart 

In [122]:
param_names = ["lambda_NS", "lambda_NSS", "lambda_NAS", "lambda_SA", "lambda_AP", "lambda_PC"]
flat_genes = ["MLH1", "MSH2", "MSH6"]

EPS = 1e-8
LAM_CAP = np.log(2.0)              # ~0.693147, annual hazard cap from P<=0.5
Z_UB = np.log(LAM_CAP)             # log upper bound used for variables in log-space

# Helpers ----
def flatten_theta(theta):
    vals = []
    for p in param_names:
        for g in flat_genes:
            vals.append(np.log(theta[p][g]))   # log transform to enforce positivity
    return np.array(vals)

def unflatten_theta(vec):
    theta_new = {k:{} for k in ["lambda_NS","lambda_NSS","lambda_NAS","lambda_SA","lambda_AP","lambda_PC","m_S0","m_A0"]}
    i = 0
    for p in param_names:
        for g in flat_genes:
            theta_new[p][g] = np.exp(vec[i])
            i += 1
    # keep misses fixed for now
    theta_new["m_S0"] = {g:0.02 for g in flat_genes}
    theta_new["m_A0"] = {g:0.01 for g in flat_genes}
    return theta_new

# Objective and constraints ----
def obj(vec):
    # get unflattened theta (hazards)
    theta = unflatten_theta(vec)
    return engel_negloglik(theta)

# --- index map consistent with flatten_theta order ---
# map (param, gene) -> index in vec (log-space)
idx = {}
k = 0
for p in param_names:
    for g in genes:
        idx[(p,g)] = k
        k += 1
n = k

# === Ordering within gene ===
# linear in z if done in log-space
# z_NSS - z_NS >= eps,  z_NAS - z_NSS >= eps
lin_constraints = []
for g in genes:
    # z_NSS - z_NS >= EPS
    a = np.zeros(n)
    a[idx[("lambda_NSS", g)]] = 1
    a[idx[("lambda_NS", g)]] = -1
    lin_constraints.append(LinearConstraint(a, lb=EPS, ub=np.inf))
    
    # z_NAS - z_NSS >= EPS
    a = np.zeros(n)
    a[idx[("lambda_NAS", g)]] = 1
    a[idx[("lambda_NSS", g)]] = -1
    lin_constraints.append(LinearConstraint(a, lb=EPS, ub=np.inf))

# === Cross-gene similarity ===
# bound pairwise differences for NSS and NAS: |z_g1 - z_g2| <= Δ  (Δ = log R)
R = 2.0               # e.g., allow up to 2x differences across genes
DELTA = np.log(R)

for p in ["lambda_NSS", "lambda_NAS"]:
    for i in range(len(genes)):
        for j in range(i+1, len(genes)):
            g1, g2 = genes[i], genes[j]
            a = np.zeros(n)
            a[idx[(p,g1)]] = 1
            a[idx[(p,g2)]] = -1
            lin_constraints.append(LinearConstraint(a, lb=-DELTA, ub=DELTA))

# === Limit NAS–NSS, NSS-NS gap within a gene ===
delta_gap = np.log(2.0) # at most 2x hazard difference
for g in genes:
    #  EPS <= z_NSS - z_NS <= delta_gap  (keeps NSS only modestly larger)
    a = np.zeros(n)
    a[idx[("lambda_NSS", g)]] = 1
    a[idx[("lambda_NS", g)]] = -1
    lin_constraints.append(LinearConstraint(a, lb=EPS, ub=delta_gap))
    
    #  EPS <= z_NAS - z_NSS <= delta_gap  (keeps NAS only modestly larger)
    a = np.zeros(n)
    a[idx[("lambda_NAS", g)]] = 1
    a[idx[("lambda_NSS", g)]] = -1
    lin_constraints.append(LinearConstraint(a, lb=EPS, ub=delta_gap))
    
# === Bounds: cap annual probability at 0.5 (hazard <= ln 2) for NS/NSS/NAS ===
LAM_CAP = np.log(2.0)       # hazard cap in ORIGINAL space (~0.693)
Z_UB = np.log(LAM_CAP)      # corresponding cap in log-space (~-0.3665)

lower, upper = [], []
for p in param_names:
    for g in genes:
        if p in ("lambda_NS", "lambda_NSS", "lambda_NAS"):
            lower.append(-np.inf)
            upper.append(Z_UB)
        else:
            lower.append(-np.inf)
            upper.append(np.inf)
bounds = Bounds(lower, upper)



# Multi-start optimization ----
def run_multistart(theta0, n_starts=20, sd_log=0.5, top_n=5):
    x0 = flatten_theta(theta0)
    results = []  # collect (loss, vector, dict)
    
    for i in range(n_starts):
        # Randomly perturb log-space parameters
        x_start = x0 + np.random.normal(0, sd_log, size=len(x0))
        
        # Optimize
        res = minimize(
            obj,
            x0,  # your log-initial vector
            method="trust-constr",           # or "SLSQP"
            constraints=lin_constraints,
            bounds=bounds,
            options={"maxiter": 5000, "gtol": 1e-10, "barrier_tol": 1e-12}
        )
        
        theta_i = unflatten_theta(res.x)
        results.append({
            "run": i + 1,
            "loss": res.fun,
            "theta": theta_i,
            "vec": res.x
        })
        print(f"Run {i+1}/{n_starts} finished  loss={res.fun:.4f}")
    
    # Sort by loss (ascending)
    results.sort(key=lambda x: x["loss"])
    
    # Keep top N
    best_runs = results[:top_n]
    
    # (Optional) summarize losses
    print("\nTop runs:")
    for r in best_runs:
        print(f"  Run {r['run']:2d} | loss={r['loss']:.4f}")
    
    return best_runs

In [111]:
runs = run_multistart(theta0, n_starts=5, sd_log=1.0)

/Users/sophiewagner/repos/lynch-syndrome/.venv/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


Run 1/5 finished  loss=-1051.7376
Run 2/5 finished  loss=-1051.7376
Run 3/5 finished  loss=-1051.7376
Run 4/5 finished  loss=-1051.7376
Run 5/5 finished  loss=-1051.7376

Top runs:
  Run  1 | loss=-1051.7376
  Run  2 | loss=-1051.7376
  Run  3 | loss=-1051.7376
  Run  4 | loss=-1051.7376
  Run  5 | loss=-1051.7376


### Inspect results

In [112]:
best_theta = runs[0]['theta']
pd.DataFrame(best_theta)

,lambda_NS,lambda_NSS,lambda_NAS,lambda_SA,lambda_AP,lambda_PC,m_S0,m_A0
MLH1,0.063180,0.083555,0.165820,0.377350,1.713386,0.403239,0.02,0.01
MSH2,0.083482,0.166859,0.331000,0.472981,0.859166,0.396963,0.02,0.01
MSH6,0.063056,0.117505,0.216291,0.234700,0.587268,0.399910,0.02,0.01


In [123]:
r_crc_py, p10_aden, p10_adv, p10_crc = simulate_engel(best_theta)

In [124]:
pd.DataFrame({
    "CRC py": r_crc_py.values(),
    "10y adenoma": p10_aden.values(),
    "10y adv adenoma": p10_adv.values(),
    "10y CRC": p10_crc.values(),
    "Sojourn time": [1.0 / best_theta["lambda_PC"][g] for g in genes],
},index=genes)

,CRC py,10y adenoma,10y adv adenoma,10y CRC,Sojourn time
MLH1,0.012882,0.322000,0.076997,0.115032,2.479920
MSH2,0.012080,0.442004,0.178013,0.108731,2.519126
MSH6,0.004781,0.384001,0.094002,0.045742,2.500563


# Poisson targets (counts with exposure)
| Item         |   MLH1           |     MSH2         | MSH6             |
|--------------|------------------|------------------|------------------|
| PY           |  12798 | 7961 | 2550 |
| CRC counts   |  167   | 93  | 12 |

# 10-year cumulative risks (value, 95% CI)
| Item         |   MLH1           |     MSH2         | MSH6             |
|--------------|------------------|------------------|------------------|
| Adenoma      | 32.2 (29.2–35.2) | 44.2 (40.0–48.4) | 38.4 (30.8–45.9) |
| Advanced     |  7.7 ( 6.0– 9.4) | 17.8 (14.6–21.0) |  9.4 ( 5.4–13.4) |
| CRC (10y)    | 11.3 ( 9.4–13.2) | 11.4 ( 8.9–14.0) |  4.7 ( 1.8– 7.7) |